In [1]:
import warnings
warnings.filterwarnings('ignore')

import json
import numpy as np
import pandas as pd
import os
import requests
import sys

import atomsci.ddm.pipeline.model_pipeline as mp
import atomsci.ddm.pipeline.parameter_parser as parse
import atomsci.ddm.utils.curate_data as curate_data
import atomsci.ddm.utils.struct_utils as struct_utils
from atomsci.ddm.pipeline import perf_plots as pp
import atomsci.ddm.utils.data_curation_functions as dcf
import atomsci.ddm.pipeline.diversity_plots as dp
import atomsci.ddm.pipeline.chem_diversity as cd

Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/jupyter/atomsci/lib/python3.8/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.
DEBUG:ATOM:Model tracker client not supported in your environment; will save models in filesystem only.


In [2]:
import deepchem as dc

In [3]:
target_name='gap'

In [5]:
ofile='gap_dtc_smiles.csv'

In [6]:
import io
url = 'https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/qm9.csv'
download = requests.get(url).content

In [7]:
orig_df = pd.read_csv(io.StringIO(download.decode('utf-8')), sep=',', header=0 )

In [9]:


# From our dataframe, we are working with the PIC50 column 
data=orig_df

# Here we are defining our response column, which will be PIC50
column = 'gap'

# tolerance: Percentage of individual respsonse values allowed to different from 
# the average to be included in averaging
tolerance = 10

# list_bad_duplicates: Print structures with bad duplicates
list_bad_duplicates = 'Yes'

# max_std: Maximum allowed standard deviation for computed average response value
# NOTE: In this example, we set this value very high to disable this feature
max_std = 1

# compound_id: Compound ID column
compound_id = 'mol_id'

# smiles_col: SMILES column
smiles_col = 'smiles'

# Here we are creating a new dataframe, called check_df
check_df = curate_data.average_and_remove_duplicates(column, tolerance, 
                                                       list_bad_duplicates, 
                                                       data, max_std, 
                                                       compound_id=compound_id, 
                                                       smiles_col=smiles_col)

Bad duplicates removed from dataset
Dataframe size (133885, 25)
List of 'bad' duplicates removed
Empty DataFrame
Columns: [mol_id, gap, VALUE_NUM_mean, Perc_Var, VALUE_NUM_std]
Index: []

Dataset de-duplicated
Dataframe size (133802, 25)
New column created with averaged values:  VALUE_NUM_mean


In [10]:
#specifies the column we want (response value) and also tells it to include the duplicates
column='gap'; #'standard_value'
list_bad_duplicates='Yes'

# Actually compiles all the data with the specifies column and tells what columns to include 
# (inchi_key, smiles, and relation)
temp_df=curate_data.aggregate_assay_data(data, 
                                         value_col=column, 
                                         output_value_col=None,
                                         label_actives=True,
                                         active_thresh=6,
                                         id_col='mol_id', 
                                         smiles_col='smiles',
                                         relation_col = 'A')

#Removes all duplicates from the data frame and puts it in the final, curated dataframe
curated_df = temp_df[~temp_df.isin([np.inf]).any(1)]

[21:56:06] Can't kekulize mol.  Unkekulized atoms: 2 6
[21:56:06] Can't kekulize mol.  Unkekulized atoms: 3 5
[21:56:07] Can't kekulize mol.  Unkekulized atoms: 3 7


TypeError: iteration over a 0-d array